In [1]:
import csv
import numpy as np
import pandas as pd
from pprint import pprint
import pymc as pm
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
pre_df = pd.read_csv('../data/Top_Traversals_demo-1daybehavior_20140401.csv', header=1)

/Users/Trent/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Data Formatting

In [4]:
pre_df.head()

,,,,,,2014-04-01
Rank,Sessions,Total Sessions,% of Total Sessions,Paths,% of Total Paths,Path
1,792522,4802851,0.16501074049559314,792522,0.16501074049559314,journey.entry->journey.exit
2,413008,4802851,0.08599225751537992,413008,0.08599225751537992,journey.entry->web.entry->webevent.login->webe...
3,283578,4802851,0.05904368051392808,283578,0.05904368051392808,journey.entry->web.entry->webevent.login->webe...
4,241868,4802851,0.05035925536728081,241868,0.05035925536728081,journey.entry->web.entry->webevent.login->webe...


In [5]:
pre_df.describe()

,2014-04-01
count,438983
unique,438983
top,journey.entry->ivr.entry->ivr.proactive balanc...
freq,1


In [6]:
paths = np.array([ 'Path'])
for row_ind in range(2, len(pre_df)-400000):
    path = list(str(pre_df.iloc[row_ind, :]).split())[1]
    paths = np.vstack((paths, path))

print paths

[['Path']
 ['journey.entry->web.entry->webevent.login->webe...']
 ['journey.entry->web.entry->webevent.login->webe...']
 ..., 
 ['journey.entry->mobile.entry->mobileevent.login...']
 ['journey.entry->mobile.entry->mobileevent.login...']
 ['journey.entry->mobile.entry->mobileevent.login...']]


In [7]:
paths = paths[1:]
for journey in range(len(paths)):
    paths[journey] = paths[journey][0].replace('->', ' ')

print paths

[['journey.entry web.entry webevent.login webe...']
 ['journey.entry web.entry webevent.login webe...']
 ['journey.entry web.entry webevent.login webe...']
 ..., 
 ['journey.entry mobile.entry mobileevent.login...']
 ['journey.entry mobile.entry mobileevent.login...']
 ['journey.entry mobile.entry mobileevent.login...']]


In [8]:
paths = np.transpose(paths)
print paths

[['journey.entry web.entry webevent.login webe...'
  'journey.entry web.entry webevent.login webe...'
  'journey.entry web.entry webevent.login webe...' ...,
  'journey.entry mobile.entry mobileevent.login...'
  'journey.entry mobile.entry mobileevent.login...'
  'journey.entry mobile.entry mobileevent.login...']]


In [9]:
Wd =  [len(paths[0][i].split()) for i in range(len(paths[0]))]
print Wd

[4, 4, 4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 4, 3, 4, 3, 3, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 3, 2, 4, 4, 3, 4, 3, 3, 3, 4, 3, 4, 3, 4, 4, 3, 4, 3, 3, 2, 3, 3, 3, 3, 4, 3, 3, 3, 3, 4, 4, 3, 4, 2, 4, 3, 4, 3, 4, 4, 4, 2, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 2, 4, 4, 4, 2, 4, 5, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 4, 3, 4, 4, 4, 2, 3, 4, 4, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 4, 2, 4, 3, 4, 3, 3, 4, 2, 4, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 3, 4, 3, 4, 4, 3, 4, 3, 4, 3, 3, 4, 4, 3, 3, 3, 4, 3, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 5, 3, 4, 3, 4, 2, 3, 4, 3, 3, 3, 4, 3, 4, 4, 3, 3, 4, 4, 3, 4, 3, 3, 4, 4, 4, 2, 4, 4, 4, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 4, 4, 3, 4, 4, 3, 4, 4, 3, 4, 4, 4, 3, 4, 4, 4, 3, 3, 3, 3, 4, 4, 3, 4, 4, 4, 3, 3, 3, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 3, 3, 4, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 3, 4, 4, 3, 4, 4, 2, 4, 3, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 3, 3, 3, 3, 4, 2, 4, 3, 4, 4, 4, 4, 3, 4, 4, 3, 3, 2, 4, 3, 3, 4, 3, 4, 4, 

## Building Tfidf Matrix

In [10]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(paths[0])
vectors.shape

(38981, 177)

In [12]:
print vectors

  (0, 94)	0.242751275625
  (0, 68)	0.48550255125
  (0, 169)	0.40011518097
  (0, 173)	0.446892379334
  (0, 98)	0.342640019179
  (0, 170)	0.477652542567
  (1, 94)	0.242751275625
  (1, 68)	0.48550255125
  (1, 169)	0.40011518097
  (1, 173)	0.446892379334
  (1, 98)	0.342640019179
  (1, 170)	0.477652542567
  (2, 94)	0.242751275625
  (2, 68)	0.48550255125
  (2, 169)	0.40011518097
  (2, 173)	0.446892379334
  (2, 98)	0.342640019179
  (2, 170)	0.477652542567
  (3, 94)	0.288731596783
  (3, 68)	0.144365798392
  (3, 54)	0.682432335206
  (3, 74)	0.655803849591
  (4, 94)	0.125444764523
  (4, 68)	0.250889529047
  (4, 88)	0.832031568202
  :	:
  (38976, 94)	0.244178482696
  (38976, 68)	0.488356965392
  (38976, 98)	0.344654501933
  (38976, 108)	0.525243710708
  (38976, 111)	0.554271288811
  (38977, 94)	0.244178482696
  (38977, 68)	0.488356965392
  (38977, 98)	0.344654501933
  (38977, 108)	0.525243710708
  (38977, 111)	0.554271288811
  (38978, 94)	0.244178482696
  (38978, 68)	0.488356965392
  (38978, 98)	

In [ ]:
vectors = vectors.todense()
print vectors

In [17]:
# V is the vocab or num of words and D is the number of documents
D, V = vectors.shape
# K is the number of topics
K = 10


alpha = np.ones(K)
beta = np.ones(V)

theta = pm.Container([pm.CompletedDirichlet("theta_%s" % i, pm.Dirichlet("ptheta_%s" % i, theta=alpha))\
                      for i in range(D)])
phi = pm.Container([pm.CompletedDirichlet("phi_%s" % k, pm.Dirichlet("pphi_%s" % k, theta=beta))\
                    for k in range(K)])

#Wd commented out because it is defined above
#Wd = [len(doc) for doc in vectors]

z = pm.Container([pm.Categorical('z_%i' % d,
                     p = theta[d],
                     size=Wd[d],
                     value= np.random.randint(K, size=Wd[d]))
                  for d in range(D)])

# cannot use p=phi[z[d][i]] here since phi is an ordinary list while z[d][i] is stochastic                                                                     
w = pm.Container([pm.Categorical("w_%i_%i" % (d,i),
                    p = pm.Lambda('phi_z_%i_%i' % (d,i),
                    lambda z=z[d][i], phi=phi: phi[z]),
                    value=vectors[d][i],
                    observed=True)
                  for d in range(D) 
                  for i in range(Wd[d])])

model = pm.Model([theta, phi, z, w])

M = pm.MCMC(model)
M.sample(5000,burn=500)
pm.Matplot.plot(M)

IndexError: index 1 is out of bounds for axis 0 with size 1